<a href="https://colab.research.google.com/github/lhsluis/TranscreveAudio/blob/main/TranscreveAudio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



*   Criado por: Luís Henrique Santos
*   Data: 09/05/2024
*   Versão: 1.0
*   Descrição: A idéia é utilizar o Gemini para transcrever áudios, principalmente os do whatsapp, futuramente quero incluir isso em um chatbot automatizado (seja no próprio whatsapp ou no telegram) onde o usuário envia o áudio e ele é devolvido transcrito.
*   Obs.: Os comentários detalhados foram inseridos diretamente no código










In [9]:
# Instalando as dependencias
!pip install -q -U google-generativeai

In [ ]:
# importando as bibliotecas necessárias
from pathlib import Path
import hashlib
import google.generativeai as genai
from google.colab import files
from IPython.display import clear_output
import os

# Não esqueça de configurar aqui sua KEY junto aos secrets do colab
from google.colab import userdata
mykey = userdata.get('GEMINI_API_KEY')

# Passando a key para a API
genai.configure(api_key=mykey)

# Função para upload do arquivo
def upload_audio():
  uploaded = files.upload()
  for filename in uploaded.keys():
    return filename

# Executa a função de upload e obtém o nome do arquivo
audio_filename = upload_audio()

# Guarda o path do arquivo que foi feito o upload
audio_path = f"/content/{audio_filename}"


# Configurando o modelo
generation_config = {
  "temperature": 0.5,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

# Setando as configurações de segurança
# Como a ideia é a transcrição de áudios informais e principalmente do dia a dia
# baixei o threshold da categoria SEXUALLY_EXPLICIT pois ele estava entendendo algumas expressões do dia a dia como impróprias.
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

# Inicializando o Modelo com as configurações do passo anterior
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

# Iniciando o chat history com o upload do arquivo escolhido que ja foi salvo
# na estrutura de pastas do colab no passo anterior
convo = model.start_chat(history=[
  {
    "role": "user",
    "parts": [genai.upload_file(audio_path)]
  }
])

# limpando o output para quando se executa repetidas vezes
clear_output()

# enviando o prompt de comando para a transcrição
convo.send_message("Transcreva o seguinte áudio, o áudio esta em português do Brasil, em sua transcrição insira uma quebra de linha sempre que chegar a um ponto final")

# escrevendo a transcrição
print(convo.last.text)

# apagando o arquivo ja transcrito
os.remove(audio_path)

Ps.: Notei que o modelo acaba aluciando com audios muito curtos, em alguns áudios de teste de 5 segundos ou menos ele acaba por criar todo um plot que não tem nada a ver com o áudio em si.
